In [2]:
import spacy
import pandas as pd
from spacy import displacy
from spacy import tokenizer

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
doc = nlp("WHEN in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature’s God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation. We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness. That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed,—That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.—Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world. He has refused his Assent to Laws, the most wholesome and necessary for the public good. He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them. He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.")
print(doc)


WHEN in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature’s God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation. We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness. That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed,—That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them s

In [5]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text, token.head.pos_)

WHEN SCONJ advmod in ADP
in ADP prep becomes VERB
the DET det Course NOUN
Course NOUN pobj in ADP
of ADP prep Course NOUN
human ADJ amod events NOUN
events NOUN pobj of ADP
, PUNCT punct becomes VERB
it PRON nsubj becomes VERB
becomes VERB ROOT becomes VERB
necessary ADJ acomp becomes VERB
for SCONJ mark dissolve VERB
one NUM nummod people NOUN
people NOUN nsubj dissolve VERB
to PART aux dissolve VERB
dissolve VERB advcl becomes VERB
the DET det bands NOUN
political ADJ amod bands NOUN
bands NOUN dobj dissolve VERB
which PRON nsubj connected VERB
have AUX aux connected VERB
connected VERB relcl bands NOUN
them PRON dobj connected VERB
with ADP prep connected VERB
another PRON pobj with ADP
, PUNCT punct dissolve VERB
and CCONJ cc dissolve VERB
to PART aux assume VERB
assume VERB conj dissolve VERB
among ADP prep assume VERB
the DET det powers NOUN
powers NOUN pobj among ADP
of ADP prep powers NOUN
the DET det earth NOUN
earth NOUN pobj of ADP
, PUNCT punct assume VERB
the DET det stati

In [6]:
data = pd.DataFrame(columns = ['token', 'pos_tag'])
for token in doc:
    data = pd.concat([data, pd.DataFrame.from_records([{'token': token.text, 'pos_tag': token.pos_}])], ignore_index=True, axis = 0)

In [7]:
print(data)

          token pos_tag
0          WHEN   SCONJ
1            in     ADP
2           the     DET
3        Course    NOUN
4            of     ADP
..          ...     ...
469  formidable     ADJ
470          to     ADP
471     tyrants    NOUN
472        only     ADV
473           .   PUNCT

[474 rows x 2 columns]


In [8]:
data_counts = data.groupby(['token', 'pos_tag']).size().reset_index(name = 'counts').sort_values(by = 'counts', ascending = False)
print(data_counts)

     token pos_tag  counts
0        ,   PUNCT      31
211    the     DET      25
157     of     ADP      21
223     to    PART      19
59     and   CCONJ      17
..     ...     ...     ...
236  while   SCONJ       1
238   will     AUX       1
240  world    NOUN       1
241  would     AUX       1
242     ’s    PART       1

[243 rows x 3 columns]


In [9]:
data_pos_counts = data.groupby(['pos_tag'])['token'].count().sort_values(ascending = False)
data_pos_counts.head(15)

pos_tag
NOUN     71
VERB     54
ADP      51
PUNCT    45
PRON     45
DET      42
ADJ      41
AUX      29
PROPN    26
PART     22
CCONJ    19
SCONJ    16
ADV      12
NUM       1
Name: token, dtype: int64

Name entity recognition

In [10]:
abraham_bio = "abraham lincoln (1809–1865) was the 16th president of the united states, widely regarded as one of the nation’s greatest leaders for guiding america through the civil war and working to end slavery. born on february 12, 1809, in a one-room log cabin in hardin county, kentucky, lincoln grew up in poverty with little formal education but developed a passion for reading and self-learning. after moving to illinois in 1830, he worked various jobs, served in the illinois militia during the black hawk war, and entered politics in 1834 as a whig, later becoming a lawyer. his debates with stephen a. douglas in 1858 brought him national recognition for his opposition to the spread of slavery. elected president in 1860 as a republican, his victory prompted southern states to secede, leading to the outbreak of the civil war in 1861. lincoln issued the emancipation proclamation in 1863, reframing the war as a fight for both union and freedom, and delivered the gettysburg address that same year, emphasizing equality and democracy. he pushed for the 13th amendment to abolish slavery and sought national reconciliation after the union’s victory. just days after the confederacy’s surrender in april 1865, lincoln was assassinated by john wilkes booth at ford’s theatre in washington, d.c. his legacy endures as a symbol of integrity, perseverance, and moral courage, remembered for preserving the union and advancing human liberty."
doc2 = nlp(abraham_bio)

In [11]:
for word in doc2.ents:
    print(word.text, word.label_)

abraham lincoln PERSON
1809–1865 DATE
16th ORDINAL
the united states GPE
february 12, 1809 DATE
one CARDINAL
hardin county GPE
kentucky GPE
lincoln ORG
illinois GPE
1830 DATE
the illinois militia FAC
1834 DATE
stephen a. douglas PERSON
1858 DATE
1860 DATE
republican NORP
1861 DATE
lincoln ORG
1863 DATE
same year DATE
the 13th DATE
just days DATE
april 1865 DATE
lincoln ORG
john wilkes PERSON
ford ORG
’s PRODUCT
washington GPE
d.c. GPE


In [13]:
displacy.render(doc2, style='ent', jupyter = True)

ImportError: cannot import name 'display' from 'IPython.core.display' (C:\Users\jonah\PycharmProjects\JupyterProject\.venv\Lib\site-packages\IPython\core\display.py)